In [ ]:
#-------------------------------------------------------------------------------
# Name:        module1
# Purpose:
#
# Author:      sravani
#
# Created:     26/11/2015
# Copyright:   (c) sravani 2015
# Licence:     <your licence>
#-------------------------------------------------------------------------------

import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model


def parseDataFromFile(fname):
  for l in open(fname):
    yield eval(l)

print "Reading data..."
data = list(parseDataFromFile("opt"))
data_copy= data[:]
train= data[:25000]
test= data[25000:]
print "done"

In [ ]:
### Ignore capitalization and remove punctuation

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
  r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

print len(wordCount)

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]
print words

### With stemming
#wordCount = defaultdict(int)
#punctuation = set(string.punctuation)
#stemmer = PorterStemmer()
#for d in data:
 # r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
  #for w in r.split():
   # w = stemmer.stem(w)
    #wordCount[w] += 1

#print len(wordCount)